In [286]:
import numpy as np
import plotly as plt

In [287]:
import plotly.express as px

In [288]:
# Parameters

# General
t = 0
T = 300
dt = T / 100
np.random.seed(100)

# Price
mu = 0
sigma = 0.1
P_0 = 45

# Spread
intensity = 1

# Cash and inventory
Y_0 = 0
X_0 = 0
delta = 0.005

# Spread

In [289]:
def poisson_point_process_sim(intensity, T):

    N = np.random.poisson(intensity * T)
    T_i = np.cumsum(np.random.exponential(1 / intensity, N))    

    return (N, T_i)

In [290]:
def spread_sim(initial_state, spread_values, T, matrix, N): 
    
    U = np.random.uniform(0, 1, 1000)
    matrix_sum = np.cumsum(matrix, axis = 1)
    s = [initial_state]
    for i in range(1, N):
        potential_position = np.sum(U[i] > matrix_sum[s[i - 1], ])
        if potential_position == s[i - 1] or potential_position == 6:
            s.append(potential_position - 1)
        else:
            s.append(potential_position)
            
    return s

In [291]:
# Spread specific objects

prob_matrix = np.array([[0, 0.41, 0.22, 0.16, 0.142, 0.065], 
                        [0.201, 0, 0.435, 0.192, 0.103, 0.067],
                        [0.113, 0.221, 0, 0.4582, 0.147, 0.059],
                        [0.07, 0.085, 0.275, 0, 0.465, 0.102],
                        [0.068, 0.049, 0.073, 0.363, 0, 0.446],
                        [0.077, 0.057, 0.059, 0.112, 0.692, 0]])
                        
spread_values = [0.005, 0.01, 0.015, 0.02, 0.025, 0.03]


In [292]:
PPP_spread = poisson_point_process_sim(intensity, T)
spread_simulated = spread_sim(0, spread_values, T, prob_matrix, PPP_spread[0])
s_t = np.array([spread_values[i] for i in spread_simulated])

In [293]:
len(s_t)

302

In [294]:
len(spread_simulated)

302

In [295]:
fig = px.line(s_t)
fig.update_traces(line_color = "maroon")
fig.update_layout(title_text = "Spread evolution", title_x = 0.5)
fig.show()

In [296]:
fig = px.line(spread_simulated)
fig.update_traces(line_color = "maroon")
fig.update_layout(title_text = "Spread evolution", title_x = 0.5)
fig.show()

In [297]:
fig = px.line(W_t)
fig.update_traces(line_color = "maroon")
fig.update_layout(title_text = "Evolution of price over time", title_x = 0.5)
fig.show()

In [298]:
fig = px.line(W_t)
fig.update_layout(title_text = "W_t", title_x = 0.5)
fig.update_layout(
    {
        "paper_bgcolor": "rgba(1, 20, 0, 1)",
        "plot_bgcolor": "rgba(1, 0, 0, 1)",
    }
)

fig.show()

# Execution of orders

In [347]:
def orders_simulaton(intensity_matrix, spread_changes, spread, lambda_max, initial_intensities, q):

    lambda_process = []

    for t, i in zip(initial_intensities, range(len(initial_intensities) + 1)):
        spread_position = np.sum(t >= spread_changes)  # checking the spread at the arrival time t
        lambda_process.append(intensity_matrix[spread[spread_position - 1], q[i]])   # choosing the realisation of intensity process based on the estimated intensity function and simulated spread


    acceptance = np.where(lambda_process / lambda_max < np.random.uniform(0, 1, len(initial_intensities)), False, True)  # accepted arrival times

    arrival_times = initial_intensities[acceptance]  # final arrival times
    q_filtered = q[acceptance]  # filtered quotes - only at accepted arrival times

    return (arrival_times, q_filtered)

In [348]:
intensity_matrix_a = np.array([[0.0539, 0.1485], 
                              [0.0465, 0.0979],
                              [0.0401, 0.0846],
                              [0.0360, 0.0856],
                              [0.0435, 0.1009],
                              [0.0554, 0.1202]]) # Estimated intensities for given quotes and spread for ask

intensity_matrix_b = np.array([[0.0718, 0.1763], 
                              [0.0520, 0.1144],
                              [0.0419, 0.0915],
                              [0.0409, 0.0896],
                              [0.0452, 0.0930],
                              [0.0614, 0.1255]])    # Estimated intensities for given quotes and spread for bid

intensity_matrix_sym = (intensity_matrix_a + intensity_matrix_b) / 2    # Making bid and ask sides symmetric

In [349]:
lambda_max = np.max(intensity_matrix_sym)  # maximum lambda serving as a reference for acceptance probability
intensity_orders_simulation = poisson_point_process_sim(np.max(lambda_max), T) # simulation of the poisson process with maximum lambda as intensity

q_a_initial = np.random.binomial(1, 0.5, len(intensity_orders_simulation[1]))  # ad-hoc choice of quotes for ask
q_b_initial = np.random.binomial(1, 0.5, len(intensity_orders_simulation[1]))  # ad-hoc choice of quotes for bid

In [350]:
orders_a_simulated = orders_simulaton(intensity_matrix_sym, PPP_spread[1], spread_simulated, lambda_max, intensity_orders_simulation[1], q_a_initial)
orders_b_simulated = orders_simulaton(intensity_matrix_sym, PPP_spread[1], spread_simulated, lambda_max, intensity_orders_simulation[1], q_b_initial)

orders_a = orders_a_simulated[0]  # final arrival times of ask orders
orders_b = orders_b_simulated[0]  # final arrival times of bid orders
q_a = orders_a_simulated[1]  # final quotes of ask orders at arrival times
q_b = orders_b_simulated[1]  # final quotes of bid orders at arrival times

# Price process

In [303]:
def price_sim(mu, P_0, t, sigma, W):
    return P_0 + mu * t + sigma * W  # Bachelier model dynamics

In [324]:
timeline = np.sort(np.concatenate((orders_a, orders_b, PPP_spread[1])))  # merged times at which there is a trade execution or change of spread
time_diff = np.diff(timeline)  # the difference between subsequent times

W_t = [0]

for t, i in zip(time_diff, range(1, len(time_diff) + 1)):
    W_t.append(W_t[i - 1] + np.random.normal(0, np.sqrt(t), 1)[0])  # simulation of Brownian motion between subsequent times (variance equal to time difference)

p_t = price_sim(mu, P_0, timeline, sigma, np.array(W_t))

# Cash and inventory

In [305]:
def inventory_sim(N_a, N_b, L_a, L_b, Y_0):

    Y = [Y_0]
    index_a = 0   # counter for sizes of ask orders 
    index_b = 0   # counter for sizes of bid orders 

    for t, index_y in zip(np.sort(np.concatenate((N_a, N_b))), range(1, len(np.concatenate((N_a, N_b)) + 1))):      # merged times of orders
        
        # check whether at a given time it is a bid or an ask order 
        if t in N_b:        
            Y.append(Y[index_y - 1] + l_b[index_b])   # if bid - add inventory
            index_b =+ 1
        else:
            Y.append(Y[index_y - 1] - l_a[index_a])   # if ask - subtract inventory
            index_a =+ 1

    return Y

In [364]:
def cash_sim(N_a, N_b, L_a, L_b, X_0, delta, q_a, q_b, s, spread_changes, p):

    l_a = np.random.uniform(0, 100, len(N_a)) # ad-hoc choice of size of order for ask side
    l_b = np.random.uniform(0, 100, len(N_b)) # ad-hoc choice of size of order for bid side

    X = [X_0]
    index_a = 0
    index_b = 0

    for t, index_x in zip(np.sort(np.concatenate((N_a, N_b))), range(1, len(np.concatenate((N_a, N_b)) + 1))):
        
        spread_position = np.sum(t >= spread_changes)  # Extracting spread at time t
        
        # check whether at a given time it is a bid or an ask order 
        if t in N_b:
            if q_b[index_b] == 1:   # Checking which quote to use
                pi_b = p - s[spread_position - 1] / 2 + delta
            else:
                pi_b = p - s[spread_position - 1] / 2
            X.append((X[index_x - 1] - pi_b * l_b[index_b])[0])    # If bid subtract cash as I bought
            index_b =+ 1
        else:
            if q_a[index_a] == 1:
                pi_a = p + s[spread_position - 1] / 2 - delta
            else:
                pi_a = p + s[spread_position - 1] / 2
            X.append((X[index_x - 1] + pi_a * l_a[index_a])[0])  # If ask add cash as I sold
            index_a =+ 1

    return X

In [365]:
l_a = np.random.uniform(0, 100, len(orders_a)) # ad-hoc choice of sizes of orders for ask side
l_b = np.random.uniform(0, 100, len(orders_b)) # ad-hoc choice of sizes of orders for bid side

In [372]:
Y = inventory_sim(orders_a, orders_b, l_a, l_b, Y_0)
X = cash_sim(orders_a, orders_b, l_a, l_b, X_0, delta, q_a, q_b, s_t, PPP_spread[1], p_t)
